In [1]:
import time
import torch
import random
import numpy as np
from torch import nn
from torch.functional import F
from matplotlib import pyplot as plt
from torchvision.transforms import v2
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader, Subset
from torchinfo import summary

In [2]:
transforms = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((32, 32))
    ]
)

In [3]:
train_dataset = FashionMNIST(root='dataset/', train=True, transform=transforms, target_transform=transforms, download=True)
test_dataset = FashionMNIST(root='dataset/', train=False, transform=transforms, target_transform=transforms, download=True)

In [4]:
class Sampling(nn.Module):
    def __init__(self):
        super(Sampling, self).__init__()

    def forward(self, z_mean, z_log_variance):
        B, C = z_mean.shape
        
        mean = 0
        std = 1
        z_std = 0.5 * torch.exp(z_log_variance)

        eps = torch.normal(mean, std, (B, C))
        eps = eps.to(device)
        z = z_mean + z_std * eps

        return z

In [5]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.enc_1 = nn.Sequential(
            nn.Conv2d(in_channels=11, out_channels=32, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
        )
        self.enc_2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
        )
        self.enc_3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
        )
        self.enc_4 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=2048, out_features=2)
        )
        self.enc_5 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=2048, out_features=2)
        )
        self.sam_1 = Sampling()

    def forward(self, x):
        x = self.enc_1(x)
        x = self.enc_2(x)
        x = self.enc_3(x)
        z_mean = self.enc_4(x)
        z_log_variance = self.enc_5(x)
        z = self.sam_1(z_mean, z_log_variance)
        
        return z_mean, z_log_variance, z

In [6]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.dec_1 = nn.Sequential(
            nn.Linear(in_features=12, out_features=2048),
            nn.Unflatten(dim=1, unflattened_size=(128, 4, 4))
        )
        self.dec_2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=3, padding=1, output_padding=1, stride=2),
            nn.ReLU()
        )
        self.dec_3 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, padding=1, output_padding=1, stride=2),
            nn.ReLU()
        )
        self.dec_4 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, padding=1, output_padding=1, stride=2),
            nn.ReLU()
        )
        self.dec_5 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.dec_1(x)
        x = self.dec_2(x)
        x = self.dec_3(x)
        x = self.dec_4(x)
        x = self.dec_5(x)
        
        return x

In [7]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(VariationalAutoencoder, self).__init__()

        self.ae_1 = encoder
        self.ae_2 = decoder

    def forward(self, x, l):
        oh = F.one_hot(l, 10)
        le = torch.unsqueeze(oh, -1)
        le = torch.unsqueeze(le, -1)
        le = le.repeat(1, 1, 32, 32)
        x = torch.cat((x, le), dim=1)
        z_mean, z_log_variance, z = self.ae_1(x)

        z = torch.cat((z, oh), dim=1)
        reconstruction = self.ae_2(z)
        
        return z_mean, z_log_variance, reconstruction

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device}')

Using cuda


In [9]:
encoder = Encoder().to(device)
decoder = Decoder().to(device)

model = VariationalAutoencoder(encoder, decoder).to(device)

In [10]:
learning_rate = 1e-2
batch_size = 512
start_epoch = 0
epochs = 25

In [11]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [12]:
class KLDivLoss(nn.Module):
    def __init__(self):
        super(KLDivLoss, self).__init__()

    def forward(self, z_log_variance, z_mean):
        return torch.mean(-0.5 * torch.sum(1 + z_log_variance - torch.pow(z_mean, 2) - torch.exp(z_log_variance), 1))

In [13]:
bce_loss = nn.BCELoss(reduction='none')
kl_div_loss = KLDivLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
def checkpoint(model, filename):
    torch.save(model.state_dict(), filename)

def resume(model, filename):
    model.load_state_dict(torch.load(filename, weights_only=True))

In [15]:
# Copied from: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
def train_loop(dataloader, model, bce_loss, kl_div_loss, optimizer):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0
    
    for batch, (image, label) in enumerate(dataloader):
        i, l = image.to(device), label.to(device)
        z_mean, z_log_variance, reconstruction = model(i, l)
        
        reconstruction_loss = torch.mean(500 * bce_loss(reconstruction, i))
        kl_loss = kl_div_loss(z_log_variance, z_mean)
        total_loss = reconstruction_loss + kl_loss
        
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        train_loss += total_loss.item()
        
    train_loss /= num_batches    
    print(f"Avg. Train loss: {train_loss:>8f}")


# Copied from: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
def test_loop(dataloader, model, bce_loss, kl_div_loss):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for batch, (image, label) in enumerate(dataloader):
            i, l = image.to(device), label.to(device)
            z_mean, z_log_variance, reconstruction = model(i, l)

            reconstruction_loss = torch.mean(500 * bce_loss(reconstruction, i))
            kl_loss = kl_div_loss(z_log_variance, z_mean)
            total_loss = reconstruction_loss + kl_loss
            test_loss += total_loss.item()
            
    test_loss /= num_batches
    print(f"Avg. Test loss: {test_loss:>8f}")

In [ ]:
# Copied from: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
if start_epoch > 0:
    resume_epoch = start_epoch - 1
    resume(model, f"checkpoints/epoch-{resume_epoch}.pth")

for t in range(start_epoch, epochs):
    t0 = time.time()
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, bce_loss, kl_div_loss, optimizer)
    test_loop(test_dataloader, model, bce_loss, kl_div_loss)
    print(f"Epoch time: {time.time() - t0}\n")
    checkpoint(model, f"checkpoints/epoch-{t}.pth")
print("Done!")

Epoch 1
-------------------------------
Avg. Train loss: 192.464712
Avg. Test loss: 171.534460
Epoch time: 12.887245416641235

Epoch 2
-------------------------------
Avg. Train loss: 167.826374
Avg. Test loss: 169.261424
Epoch time: 12.721989154815674

Epoch 3
-------------------------------
Avg. Train loss: 166.011442
Avg. Test loss: 166.328666
Epoch time: 12.874197483062744

Epoch 4
-------------------------------
Avg. Train loss: 165.244310
Avg. Test loss: 166.114014
Epoch time: 12.858962297439575

Epoch 5
-------------------------------
Avg. Train loss: 164.665600
Avg. Test loss: 165.730865
Epoch time: 13.096653461456299

Epoch 6
-------------------------------
Avg. Train loss: 164.410780
Avg. Test loss: 165.780692
Epoch time: 12.775063753128052

Epoch 7
-------------------------------
Avg. Train loss: 164.229046
Avg. Test loss: 165.256458
Epoch time: 12.746533632278442

Epoch 8
-------------------------------
Avg. Train loss: 164.114797
Avg. Test loss: 165.280851
Epoch time: 12.5

In [ ]:
subset_indices = [random.randint(0, test_dataset.__len__() - 1) for i in range(10)]
data_subset = Subset(test_dataset, subset_indices)
data_subset_loader = DataLoader(data_subset, batch_size=1, shuffle=True)

In [ ]:
def infer_and_plot_results(dataloader, model):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    with torch.no_grad():
        for batch, (image, label) in enumerate(dataloader):
            i, l = image.to(device), label.to(device)
            z_mean, z_log_variance, reconstruction = model(i, l)

            fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
            fig.supylabel(f'Test Output')
            fig.tight_layout()
            
            ax1.imshow(torch.permute(reconstruction[0], (1, 2, 0)).cpu().data.numpy(), cmap='gray')
            ax1.set_title('Prediction')
            ax2.imshow(torch.permute(i[0], (1, 2, 0)).cpu().data.numpy(), cmap='gray')
            ax2.set_title('Reference')

In [ ]:
infer_and_plot_results(data_subset_loader, model)

In [ ]:
def visualize_latent_space(encoder):
    embeddings = []
    
    for _, (i, l) in enumerate(test_dataloader):
        i = i.to(device)
        l = l.to(device)
        oh = F.one_hot(l, 10)
        le = torch.unsqueeze(oh, -1)
        le = torch.unsqueeze(le, -1)
        le = le.repeat(1, 1, 32, 32)
        x = torch.cat((i, le), dim=1)
        
        z_mean, z_log_variance, z = encoder(x)
        z = z.cpu().data.numpy()
        embeddings.append(z)

    embeddings = np.concatenate(embeddings)
    
    plt.figure(figsize=(8, 8))
    plt.scatter(x=embeddings[:, 0], y=embeddings[:, 1], s=1, alpha=0.5, c='black')
    plt.show()

In [ ]:
visualize_latent_space(encoder)

In [ ]:
def decode_random_sample(decoder):
    embedding = torch.abs(torch.randn((1, 2)) * 1).to(device)
    l = torch.tensor([0]).to(device)
    oh = F.one_hot(l, 10)
    x = torch.cat((embedding, oh), dim=1)
   
    p = decoder(x)
    p = p[0].permute((1, 2, 0)).cpu().data.numpy()
    
    plt.figure(figsize=(1, 1))
    plt.imshow(p, cmap='gray')
    plt.show()

In [ ]:
decode_random_sample(decoder)